In [ ]:
import numpy as np
import pandas as pd

from xgboost import XGBRegressor

#### 预处理

In [23]:
dftest = pd.read_csv("test_item.csv")
dfweather = pd.read_csv("store_weather.csv")
dfprom = pd.read_csv("sku_prom.csv")
dfprice = pd.read_csv("sku_price_and_status.csv")

In [ ]:
# 添加channel
dft = pd.concat([dftest, dftest], ignore_index=True)
dft['channel'] = [1, 2] * (len(dftest)//2) + [2, 1] * (len(dftest)//2)
dft = dft[['date', 'store_id', 'sku_id', 'channel']]
dft = dft.sort_values(by=['date', 'store_id', 'sku_id'], ascending=[True, True, True], ignore_index=True)


In [ ]:
# Weather
dft1 = pd.merge(dft, dfweather[['date', 'store_id', 
                                'weather_type', 'min_temperature', 
                                'max_temperature']], 
                on=['date', 'store_id'], how='left')

In [ ]:
# Price
dft2 = pd.merge(dft1, 
                dfprice[['store_id', 'sku_id', 'date', 
                         'original_price']], 
                on=['store_id', 'sku_id', 'date'], how='left')

In [ ]:
dfprom['curr_day_ratio'] = dfprom['curr_day']/dfprom['total_days']

In [ ]:
dft3 = pd.merge(dft2, 
                dfprom[['store_id', 'sku_id', 'date', 'channel', 
                         'threshold', 'discount_off', 
                         'curr_day_ratio', 'promotion_type']], 
                on=['store_id', 'sku_id', 'date', 'channel'], how='left')
dft3[['threshold', 'discount_off', 
      'curr_day_ratio', 'promotion_type']] = dft3[['threshold', 'discount_off', 
                                                   'curr_day_ratio', 'promotion_type']].fillna(0)


In [ ]:
dft3.to_csv("sku_test.csv", index = False)

### 预测

In [11]:
import numpy as np
import pandas as pd

from xgboost import XGBRegressor
# 处理为读入格式
dft = pd.read_csv("./sku_test.csv")
def get_month_year(df):
    df['month'] = df.date.apply(lambda x: x.split('-')[1])
    df['year'] = df.date.apply(lambda x: x.split('-')[0])
get_month_year(dft)
dft['date'] = pd.to_datetime(dft['date'])
dft['day'] = dft['date'].dt.day_name()
# dft = dft.drop(['date', 'year'], axis=1)
dft.columns

Index(['date', 'store_id', 'sku_id', 'channel', 'weather_type',
       'min_temperature', 'max_temperature', 'original_price', 'threshold',
       'discount_off', 'curr_day_ratio', 'promotion_type', 'month', 'year',
       'day'],
      dtype='object')

In [12]:
freq_c1 = np.load('../train/c1_freq.npy'); freq_c2 = np.load('../train/c2_freq.npy')
grouped_dft = dft.groupby('channel')
dft_c1 = grouped_dft.get_group(1).drop('channel', axis = 1)
dft_c2 = grouped_dft.get_group(2).drop('channel', axis = 1)

In [13]:
fc1_1 = freq_c1[0][0:111]; fc1_2 = freq_c1[0][111:293]; fc1_3 = freq_c1[0][293:1000]
fc2_1 = freq_c2[0][0:111]; fc2_2 = freq_c2[0][111:293]; fc2_3 = freq_c2[0][293:1000]

dft_c1_1 = dft_c1[dft_c1['sku_id'].isin(fc1_1)].reset_index(drop=True)
dft_c1_2 = dft_c1[dft_c1['sku_id'].isin(fc1_2)].reset_index(drop=True)
dft_c1_3 = dft_c1[dft_c1['sku_id'].isin(fc1_3)].reset_index(drop=True)
dft_c2_1 = dft_c2[dft_c2['sku_id'].isin(fc2_1)].reset_index(drop=True)
dft_c2_2 = dft_c2[dft_c2['sku_id'].isin(fc2_2)].reset_index(drop=True)
dft_c2_3 = dft_c2[dft_c2['sku_id'].isin(fc2_3)].reset_index(drop=True)

In [ ]:
# one-hot encoding
dummy_variables = ['store_id', 'sku_id', 'month', 'day', 'promotion_type', 'weather_type']
for var in dummy_variables:
    dummy = pd.get_dummies(dft_c2_1[var], prefix = var, drop_first = False)
    dft_c2_1 = pd.concat([dft_c2_1, dummy], axis = 1)
dft_c2_1 = dft_c2_1.drop(dummy_variables, axis = 1)

In [ ]:
with open('../train/col_c2_1.txt', 'r') as file:
    columns = [line.strip() for line in file]
# 筛选出在 DataFrame 中不存在的新字段
new_columns = [col for col in columns if col not in dft_c2_1.columns]
# 将新字段添加到 DataFrame，并设置新列的值为 False
for new_column in new_columns:
    dft_c2_1[new_column] = False
dft_c2_1 = dft_c2_1.reindex(columns=columns)


In [ ]:
model_c2_1 = XGBRegressor()
model_c2_1.load_model('../train/xgboost_c2_1.pth')

In [ ]:
pred_c2_1 = model_c2_1.predict(dft_c2_1)
pred_c2_1 = np.round(pred_c2_1, decimals = 1)
pred_c2_1[pred_c2_1 < 0] = 0

In [10]:
dft_c1_1 = dft_c1[dft_c1['sku_id'].isin(fc1_1)].reset_index(drop=True)
dft_c1_2 = dft_c1[dft_c1['sku_id'].isin(fc1_2)].reset_index(drop=True)
dft_c1_3 = dft_c1[dft_c1['sku_id'].isin(fc1_3)].reset_index(drop=True)
dft_c2_1 = dft_c2[dft_c2['sku_id'].isin(fc2_1)].reset_index(drop=True)
dft_c2_2 = dft_c2[dft_c2['sku_id'].isin(fc2_2)].reset_index(drop=True)
dft_c2_3 = dft_c2[dft_c2['sku_id'].isin(fc2_3)].reset_index(drop=True)

In [ ]:
pred_sales_c2_1 = pd.DataFrame()
pred_sales_c2_1[['store_id', 'sku_id']] = dft_c2_1[['store_id', 'sku_id']]
pred_sales_c2_1['channel'] = 1
pred_sales_c2_1['quantity'] = pred_c2_1
pred_sales_c2_1.to_csv("pred_sales_c2_1.csv", index=False)

### 合并

In [ ]:
import pandas as pd 

In [ ]:
c1_1 = pd.read_csv("pred_sales_c1_1.csv")
c1_2 = pd.read_csv("pred_sales_c1_2.csv")
c1_3 = pd.read_csv("pred_sales_c1_3.csv")
c2_1 = pd.read_csv("pred_sales_c2_1.csv")
c2_2 = pd.read_csv("pred_sales_c2_2.csv")
c2_3 = pd.read_csv("pred_sales_c2_3.csv")


In [16]:
c1_1['date'] = dft_c1_1['date']
c1_2['date'] = dft_c1_2['date']
c1_3['date'] = dft_c1_3['date']
c2_1['date'] = dft_c2_1['date']
c2_2['date'] = dft_c2_2['date']
c2_3['date'] = dft_c2_3['date']
c1_1 = c1_1[['date', 'store_id', 'sku_id', 'channel', 'quantity']]
c1_2 = c1_2[['date', 'store_id', 'sku_id', 'channel', 'quantity']]
c1_3 = c1_3[['date', 'store_id', 'sku_id', 'channel', 'quantity']]
c2_1 = c2_1[['date', 'store_id', 'sku_id', 'channel', 'quantity']]
c2_2 = c2_2[['date', 'store_id', 'sku_id', 'channel', 'quantity']]
c2_3 = c2_3[['date', 'store_id', 'sku_id', 'channel', 'quantity']]


In [20]:
c2_1['channel'] = 2
c2_2['channel'] = 2
c2_3['channel'] = 2

In [21]:
pred_result = pd.concat([c1_1, c1_2, c1_3, c2_1, c2_2, c2_3], ignore_index=True)

In [27]:
pred_dup = pred_result[pred_result.duplicated(subset=['date', 'store_id', 'sku_id', 'channel'], keep=False)]
average_values = pred_dup.groupby(['date', 'store_id', 'sku_id', 'channel'])['quantity'].mean().reset_index()

In [28]:
merged_df = pd.merge(pred_result, average_values, on=['date', 'store_id', 'sku_id', 'channel'], 
                     how='left', suffixes=('', '_average'))
merged_df['quantity'] = merged_df['quantity_average'].combine_first(merged_df['quantity'])
merged_df = merged_df.drop(columns=['quantity_average'])

In [32]:
pred_result_nr = merged_df.drop_duplicates()
pred_result_nr = pred_result_nr.sort_values(by=['date', 'store_id', 'sku_id', 'channel'], ignore_index = False)


In [ ]:
pred_result_nr.to_csv("pred_sales.csv", index = False)